<a href="https://colab.research.google.com/github/AlexAlexeyuk/Hospital_epicrisis_data_analysis/blob/main/Parsed_data_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os
import pandas as pd
import glob
import re

In [32]:
class Utils():
    root_directory = os.chdir('/content/Untitled Folder')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    # for i in trash: 
    #     list_of_files.remove(i)


In [33]:
list_of_files = Utils.list_of_files

In [34]:
plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []

In [35]:
def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'


In [36]:
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', '').replace('ЛФК', '').replace('ФТЛ', ''))

In [37]:
def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'

In [38]:
pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:', "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_=0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+30]

In [39]:
def crp(file_):
  """returns all crp values"""
  pattern = re.sub(r'[ЦСC]РБ', 'С-реактивныйбелок', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)(\d*.\d+|\d+)')
  pattern_2 = pattern_1.findall(''.join(pattern.split()))
  try:
    return pattern_2
  except:
    return 'None'

In [40]:
def gender(file_):
  patt1 = re.compile(r'(?<=Ф.И.О:)[А-Я]\w{,20}[А-Я]\w{,20}(вич)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'male'
    else:
      return 'female'
  except:
      pass

In [41]:
def ldh(file_):
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [42]:
def cre(file_):
  """Returns all creatinine values"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d\d\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  patt1 = re.compile(r'(?<=cre)\d{,3}')
  patt2 = patt1.findall(''.join(file_.split()))
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [43]:
def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(''.join(patt1.split()))
  try:
    if all_hgb:
      return all_hgb
    else:
      return 'None'
  except:
    pass

In [44]:
def wbc(file_):
  """returns list of str with level of wbc"""
  file_ = file_.lower()
  patt1 = re.sub(r'[():]', '', file_)
  patt1 = re.sub(r'10(\*|[еe])9', '', patt1)
  patt1 = re.sub(r'\Dбщийанализкрови', 'оак', patt1) 
  patt2 = re.sub(r'(?<=оак)\d\d.\d\d.\d{2,4}|(?<=оак)\d\d.\d\d', '', patt1)
  patt3 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*.\d+|\d+)')
  all_wbc = patt3.findall(''.join(patt2.split()))
  try:
    if all_wbc:
      return all_wbc
    else:
      return 'None'
  except:
    pass

In [45]:
def plt(file_):
  """returns list of str with level of plt"""
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  patt2 = re.compile(r'(?:\wромбоцит\w|тр)')
  patt1 = re.sub(patt2, '', file_)
  patt1 = ''.join(patt1.split())
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(patt1)
  try:
    if all_plt:
      return all_plt
    else:
      return 'None'
  except:
    pass

In [46]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(''.join(file_.split())))
                sex_.append(gender(''.join(file_.split())))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
    except:
        print(file)

In [48]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [30]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')

In [49]:
data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
                                  'Treatment':tr, 'Birthday': bir, 
            'Admission':ad,
            'Discharge':dis, 'ID':ID_, 'Diagnosis': diagnosis_cleaned}

In [50]:
dataFrame = pd.DataFrame(data_dct)


In [51]:
dataFrame.head()

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis
0,[261],[8.91],[136],None,None,female,[6.2],"дексаметазон , теофил , амброксол , лазартан ...",18:11:1968,12:02:2020,21:02:2020,985,Внегоспитальная пневмония в в/д справа средней...
1,None,[1.77],[107],[86],None,female,[96],"левофлоксацин, аспикард.дезинтоксикац тер. оме...",23:06:1951,12:09:2017,15:09:2017,6026,J44 Другая хроническая обструктивная легочная ...
2,[384],[6.03],[159],None,None,male,[6],"амброксол , флустоп , теофил , цкфтриаксон ФТ",30:08:1989,04:04:2019,12:04:2019,2141,Внегоспитальная пневмония в н/д слева средней ...
3,"[250, 431]","[4.38, 6.62]","[149, 145]",[68],[325],female,"[17.42, 6, 6]","флустоп, амброксол левофлоксацин инспирон,",22:01:1973,30:11:2017,08:12:2017,7970,J18 Пневмония без уточнения возбудителя внегос...
4,"[162, 212]","[8.7, 8.82]","[165, 161]",None,None,male,[],"цефтриаксон, левофлоксацин,амброксол.",24:12:1956,04:04:2020,14:04:2020,2015,поступлении: Внегоспитальная пневмония в нижне...
